<a href="https://colab.research.google.com/github/taunoe/jupyter-notebooks/blob/main/Tekst_vektoresitused.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tekst: vektoresitused
*Praktilise näite autor: Mark Fišel*

Kaks lähenemist sõnade tähenduste modelleerimisele: käsitsi koostatud semantiline võrk WordNet ja automaatselt õpitud word2vec vektorid. EstNLTK 1.6 ei toeta veel eestikeelset WordNet'i ja Google Colab ei toeta EstNLTK 1.4 :-( seega, tutvustame siin vektoresitusi, mida õpib word2vec >:-)

Kõigepealt laeme alla sama tekstikorpuse nagu 6. ülesandes. See sisaldab ainult toorteksti ilma mingisuguste märgenditeta.

In [1]:
!wget http://data.statmt.org/wmt18/translation-task/news.2015.et.shuffled.deduped.gz
!gunzip news.2015.et.shuffled.deduped.gz
!wc -lw news.2015.et.shuffled.deduped

--2021-04-14 06:07:06--  http://data.statmt.org/wmt18/translation-task/news.2015.et.shuffled.deduped.gz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44528040 (42M) [application/x-gzip]
Saving to: ‘news.2015.et.shuffled.deduped.gz’

news.2015.et.shuffl 100%[===================>]  42.46M  13.8MB/s    in 3.1s    

2021-04-14 06:07:09 (13.8 MB/s) - ‘news.2015.et.shuffled.deduped.gz’ saved [44528040/44528040]

   962663  13002539 news.2015.et.shuffled.deduped


Seejärel loeme korpuse sisse. Sellel korral väldime EstNLTK sõnestamist, kuna see on ajamahukam ja sõnavektorite loomisel pole algvorme vaja. Seega eemaldame ainult komad, punktid jm punktuatsiooni. Lisaks teisendame kõik tähed alumisse registrisse, näiteks "Jaagu" asemel jääb väikese tähega "jaagu".

Lõpuks vaatame mitu lauset meil treenimishulgas on.

In [2]:
def lihtne_sõnastaja(lause):
  return [sõne.lower().strip('.,;:-"\'!?') for sõne in lause.strip().split()]

with open('news.2015.et.shuffled.deduped', 'r') as tekstifail:
  treenimishulk = [lihtne_sõnastaja(lause) for lause in tekstifail]

print(len(treenimishulk), "lauset")

962663 lauset


Seejärel treenime word2vec mudeli. See on implementeeritud teegis "gensim". Treenimise protsess võtab umbes 7 minutit.

In [3]:
from datetime import datetime
from gensim.models import Word2Vec

print("Algus:", datetime.now())

mudel = Word2Vec(sentences=treenimishulk[:1000000], sg=1) # ainult 1 milj. lauset, et se protsess liiga kaua ei kestaks

print("Lõpp: ", datetime.now())

Algus: 2021-04-14 06:07:35.029814
Lõpp:  2021-04-14 06:13:59.929252


Kõik! Word2vec mudel on valmis. Nüüd saame õpitud vektoresitusi vaadates võrrelda, kui sarnased sõnad omavahel on. Tuletame meelde, et ainus infoallikas vektoresituste õppimiseks oli tavaline tekst eestikeelsetest uudistest ja et need mudelid defineerivad sarnasust kui samas kontekstis (samade naabersõnade läheduses) esinemist.

**Esimene harjutus:** vaatame välja valitud sõnapaaride sarnasusi!

In [4]:
print("kuninga ja röövli sarnasus:", mudel.wv.similarity("kuningas", "röövel"))
print("kuninga ja kuninganna sarnasus:", mudel.wv.similarity("kuningas", "kuninganna"))

print("")
paarid = """kass kiisu
kass kassipoeg
kass taim
kass tubli
koer tubli
laud laua
laud laul"""

for paar in paarid.split("\n"):
  üks_sõna, teine_sõna = paar.split(" ")
  print(üks_sõna, teine_sõna, mudel.wv.similarity(üks_sõna, teine_sõna))

print("")

kuninga ja röövli sarnasus: 0.51674026
kuninga ja kuninganna sarnasus: 0.7096328

kass kiisu 0.7609651
kass kassipoeg 0.7572082
kass taim 0.57038826
kass tubli 0.38769445
koer tubli 0.37627256
laud laua 0.5160027
laud laul 0.36953387



Seejuures on oluline aru saada, et tegu on empiirilise sarnasuse hinnanguga, mis ei pruugi täpselt ühtida meie arusaamaga sünonüümidest või sõnade sarnasusest üldiselt.

**Teine harjutus:** leiame ette antud sõna kõige lähimad "naabrid" ehk nendega kõige sarnasemad sõnad (vastavalt word2vec mudelile).

In [5]:
def kuva_sarnaseimad(mudel, sõna = None, vektor = None, mitu = 13):
  if vektor is not None:
    topn = mudel.similar_by_vector(vektor, topn=mitu)
  else:
    topn = mudel.similar_by_word(sõna, topn=mitu)

  print("Sõnaga '{0}' {1} sarnaseimat sõna:".format(sõna, mitu))
  print("\n".join([sarnane_sõna + ": " + str(sarnasus) for sarnane_sõna, sarnasus in topn]))
  print("")

kuva_sarnaseimad(mudel, sõna = "mõte", mitu = 5)

for sõna in "kass koer armastus värv telefon sai".split():
  kuva_sarnaseimad(mudel, sõna = sõna, mitu = 5)

Sõnaga 'mõte' 5 sarnaseimat sõna:
idee: 0.7537893056869507
säde: 0.7470536231994629
paralleel: 0.7390496730804443
uitmõte: 0.7383795976638794
ajend: 0.7240839004516602

Sõnaga 'kass' 5 sarnaseimat sõna:
koer: 0.8484876751899719
isane: 0.798509955406189
lapselaps: 0.7614974975585938
kiisu: 0.7609650492668152
kassipoeg: 0.7572082281112671

Sõnaga 'koer' 5 sarnaseimat sõna:
kass: 0.8484877347946167
isane: 0.8186787366867065
kiisu: 0.8067450523376465
loom: 0.804841160774231
pruut: 0.78680419921875

Sõnaga 'armastus' 5 sarnaseimat sõna:
kirg: 0.8033260107040405
kiindumus: 0.7593796253204346
vaim: 0.7496028542518616
taevalik: 0.7489062547683716
kurbus: 0.7482450604438782

Sõnaga 'värv' 5 sarnaseimat sõna:
jumestuskreem: 0.8213196992874146
aroom: 0.7986755967140198
oranž: 0.7984172701835632
liturgiline: 0.7956861853599548
värvus: 0.7928823828697205

Sõnaga 'telefon' 5 sarnaseimat sõna:
mobiil: 0.7627392411231995
mobiiltelefon: 0.7473754286766052
pastakas: 0.7355202436447144
ämber: 0.728389382

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """


Kui proovida mitmetähenduslike sõnade vektoreid (nt "sai"), siis ilmneb kohe üks probleem fikseeritud vektoresitustega: nimelt on igale sõnale ainult üks vektor, mis peab katma kõiki selle sõna tähendusi. Sõna "sai" puhul tundub, et selle vektor esindab rohkem tegusõna "saama" kui pagaritoodet.

**Kolmas harjutus:** osutub, et lisaks sarnasuse umbkaudsele modelleerimisele alluvad word2vec vektorid imelikule kuid lõbusale aritmeetikale. Nimelt, mis juhtub, kui nt sõna "ilves" vektorist lahutada maha "eesti", kuid liita "ukraina"? Sarnaseimate sõnade nimekirjas leiame Ukraina presidendi nime.  Sellise "aritmeetika" oodatav vastus ei ole alati esimene, kuid satub sageli top-5 hulka. 

Teine lõbus näide on, et "prints - mees + naine" on "printsess" :-D 

Mõelge ise välja mõned näited, kus see aritmeetika töötada võiks, ning proovige neid järgneva koodi modifitseerimise abil.

In [6]:
kuva_sarnaseimad(mudel, mitu = 5, vektor = mudel.wv['ilves'] - mudel.wv['eesti'] + mudel.wv['ukraina'], sõna = "ilves-eesti+ukraina")
kuva_sarnaseimad(mudel, mitu = 5, vektor = mudel.wv['prints'] - mudel.wv['mees'] + mudel.wv['naine'], sõna = "prints-mees+naine")
kuva_sarnaseimad(mudel, mitu = 5, vektor = mudel.wv['tallinn'] - mudel.wv['eesti'] + mudel.wv['läti'], sõna = "tallinn-eesti+läti")

Sõnaga 'ilves-eesti+ukraina' 5 sarnaseimat sõna:
putin: 0.7550578117370605
ilves: 0.7533096671104431
petro: 0.7522073984146118
porošenko: 0.7507251501083374
riigipea: 0.7196485996246338

Sõnaga 'prints-mees+naine' 5 sarnaseimat sõna:
prints: 0.9505405426025391
printsess: 0.834510326385498
hertsoginna: 0.8296248316764832
williami: 0.815324068069458
catherine: 0.8020913004875183

Sõnaga 'tallinn-eesti+läti' 5 sarnaseimat sõna:
tallinn: 0.7851227521896362
riia: 0.6587557196617126
stockholm: 0.6114889979362488
pariis: 0.6047160625457764
oslo: 0.5991486310958862



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Kõik! Soovi korral võite muidugi proovida, kuidas töötab sama lähenemine teiste korpuste puhul, asendades tekstikorpuse URL-i harjutuse alguses järgmistega, näiteks:
* [eestikeelsed uudised aastast 2017](http://data.statmt.org/wmt18/translation-task/news.2017.et.shuffled.deduped.gz)
* [venekeelsed uudised](http://data.statmt.org/wmt17/translation-task/news.2016.ru.shuffled.gz)
* [europarlamendi istungite soomekeelsed protokollid](http://statmt.org/wmt15/europarl-v8.fi.tgz)
* [samade istungite inglisekeelsed protokollid](http://statmt.org/wmt14/training-monolingual-europarl-v7/europarl-v7.en.gz)

Edu!